In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.mit_bih import MITBIH
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss, ClassificationKNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 32
tree_depth = 10
batch_size = 512
device = 'cpu'
train_data_path = r'F:\Downloads\archive\mitbih_train.csv'
test_data_path = r'F:\Downloads\archive\mitbih_test.csv'

In [3]:
train_data_iter = torch.utils.data.DataLoader(MITBIH(train_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True,
                                             drop_last=True)

test_data_iter = torch.utils.data.DataLoader(MITBIH(test_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True)

In [4]:
class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)
        
    def forward(self, x):
        y = x
        y = self.conv1(y)
        y = self.relu1(y)
        y = self.conv2(y)
        y = y + x
        y = self.relu2(y)
        y = self.pool(y)
        return y


class ECGModel(nn.Module):
    def __init__(self):
        super(ECGModel, self).__init__()
        self.conv = nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=1)
        self.block1 = ConvBlock()
        self.block2 = ConvBlock()
        self.block3 = ConvBlock()
        self.block4 = ConvBlock()
        self.block5 = ConvBlock()
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 5)

    def forward(self, x, return_interm=False):
        x = self.conv(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        interm = x.flatten(1)
        x = self.fc1(interm)
        x = self.relu(x)
        x = self.fc2(x)
        
        if return_interm:
            return x, interm
        
        return x

In [5]:
knn_crt = ClassificationKNNLoss(k=k).to(device)

def train(model, loader, optimizer, device):
    model.train()

    total_loss = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        outputs, interm = model(batch, return_interm=True)
        mse_loss = F.cross_entropy(outputs, target)
        mse_loss = mse_loss.sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(interm, target)
            if torch.isinf(knn_loss):
                knn_loss = torch.tensor(0)
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(loader)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | CLS Loss: {mse_loss.item()}")

    return total_loss / len(loader)

@torch.no_grad()
def test(model, loader, device):
    model.eval()
    
    correct = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        y_pred = model(batch).argmax(dim=-1)
        correct += y_pred.eq(target.view(-1).data).sum()
    
    return correct / len(loader.dataset)

In [6]:
epochs = 200
lr = 1e-3
log_every = 10

model = ECGModel()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_params = sum([p.numel() for p in model.parameters()])
print(f'#Params: {num_params}')

#Params: 53957


In [7]:
best_valid_acc = 0
losses = []
train_accs = []
val_accs = []
for epoch in range(1, epochs + 1):
    loss = train(model, train_data_iter, optimizer, device)
#     print(f"Loss: {loss} =============================")
    losses.append(loss)
    train_acc = test(model, train_data_iter, device)
    train_accs.append(train_acc)
    valid_acc = test(model, test_data_iter, device)
    val_accs.append(valid_acc)
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, '
              f'Best: {best_valid_acc:.4f}')

Epoch 1 / 200 | iteration 0 / 171 | Total Loss: 7.587115287780762 | KNN Loss: 5.718296527862549 | CLS Loss: 1.8688185214996338
Epoch 1 / 200 | iteration 10 / 171 | Total Loss: 5.539576053619385 | KNN Loss: 4.427477836608887 | CLS Loss: 1.1120980978012085
Epoch 1 / 200 | iteration 20 / 171 | Total Loss: 4.7229814529418945 | KNN Loss: 3.9933700561523438 | CLS Loss: 0.729611337184906
Epoch 1 / 200 | iteration 30 / 171 | Total Loss: 4.43195915222168 | KNN Loss: 3.8444793224334717 | CLS Loss: 0.5874797105789185
Epoch 1 / 200 | iteration 40 / 171 | Total Loss: 4.483575820922852 | KNN Loss: 3.8387410640716553 | CLS Loss: 0.6448348164558411
Epoch 1 / 200 | iteration 50 / 171 | Total Loss: 4.415455341339111 | KNN Loss: 3.8455843925476074 | CLS Loss: 0.5698708295822144
Epoch 1 / 200 | iteration 60 / 171 | Total Loss: 4.365846157073975 | KNN Loss: 3.7997422218322754 | CLS Loss: 0.5661040544509888
Epoch 1 / 200 | iteration 70 / 171 | Total Loss: 4.351258277893066 | KNN Loss: 3.831059455871582 | CL

Epoch 4 / 200 | iteration 80 / 171 | Total Loss: 3.8422582149505615 | KNN Loss: 3.6666877269744873 | CLS Loss: 0.17557038366794586
Epoch 4 / 200 | iteration 90 / 171 | Total Loss: 3.843769073486328 | KNN Loss: 3.654789686203003 | CLS Loss: 0.1889793574810028
Epoch 4 / 200 | iteration 100 / 171 | Total Loss: 3.826467990875244 | KNN Loss: 3.6591689586639404 | CLS Loss: 0.16729910671710968
Epoch 4 / 200 | iteration 110 / 171 | Total Loss: 3.8243889808654785 | KNN Loss: 3.6761441230773926 | CLS Loss: 0.1482449471950531
Epoch 4 / 200 | iteration 120 / 171 | Total Loss: 3.7941391468048096 | KNN Loss: 3.6541688442230225 | CLS Loss: 0.1399703025817871
Epoch 4 / 200 | iteration 130 / 171 | Total Loss: 3.7871880531311035 | KNN Loss: 3.6518688201904297 | CLS Loss: 0.13531926274299622
Epoch 4 / 200 | iteration 140 / 171 | Total Loss: 3.8234703540802 | KNN Loss: 3.676626443862915 | CLS Loss: 0.14684398472309113
Epoch 4 / 200 | iteration 150 / 171 | Total Loss: 3.854278326034546 | KNN Loss: 3.675780

Epoch 7 / 200 | iteration 160 / 171 | Total Loss: 3.7192060947418213 | KNN Loss: 3.6243038177490234 | CLS Loss: 0.09490218758583069
Epoch 7 / 200 | iteration 170 / 171 | Total Loss: 3.69795560836792 | KNN Loss: 3.6322481632232666 | CLS Loss: 0.06570734828710556
Epoch: 007, Loss: 3.7457, Train: 0.9769, Valid: 0.9742, Best: 0.9742
Epoch 8 / 200 | iteration 0 / 171 | Total Loss: 3.70446515083313 | KNN Loss: 3.6155078411102295 | CLS Loss: 0.08895742148160934
Epoch 8 / 200 | iteration 10 / 171 | Total Loss: 3.734149217605591 | KNN Loss: 3.6574413776397705 | CLS Loss: 0.07670783251523972
Epoch 8 / 200 | iteration 20 / 171 | Total Loss: 3.722667694091797 | KNN Loss: 3.6310293674468994 | CLS Loss: 0.09163843840360641
Epoch 8 / 200 | iteration 30 / 171 | Total Loss: 3.700176477432251 | KNN Loss: 3.6133313179016113 | CLS Loss: 0.08684512227773666
Epoch 8 / 200 | iteration 40 / 171 | Total Loss: 3.714923620223999 | KNN Loss: 3.64667010307312 | CLS Loss: 0.06825359165668488
Epoch 8 / 200 | iterati

Epoch 11 / 200 | iteration 50 / 171 | Total Loss: 3.685824394226074 | KNN Loss: 3.596081018447876 | CLS Loss: 0.08974327892065048
Epoch 11 / 200 | iteration 60 / 171 | Total Loss: 3.7582485675811768 | KNN Loss: 3.6727967262268066 | CLS Loss: 0.08545186370611191
Epoch 11 / 200 | iteration 70 / 171 | Total Loss: 3.6923916339874268 | KNN Loss: 3.6167309284210205 | CLS Loss: 0.07566066086292267
Epoch 11 / 200 | iteration 80 / 171 | Total Loss: 3.669750213623047 | KNN Loss: 3.6241214275360107 | CLS Loss: 0.04562876373529434
Epoch 11 / 200 | iteration 90 / 171 | Total Loss: 3.686194658279419 | KNN Loss: 3.5548713207244873 | CLS Loss: 0.13132326304912567
Epoch 11 / 200 | iteration 100 / 171 | Total Loss: 3.7228708267211914 | KNN Loss: 3.6078102588653564 | CLS Loss: 0.11506067216396332
Epoch 11 / 200 | iteration 110 / 171 | Total Loss: 3.689363718032837 | KNN Loss: 3.6066689491271973 | CLS Loss: 0.08269474655389786
Epoch 11 / 200 | iteration 120 / 171 | Total Loss: 3.7255859375 | KNN Loss: 3.6

Epoch 14 / 200 | iteration 120 / 171 | Total Loss: 3.656766891479492 | KNN Loss: 3.604492425918579 | CLS Loss: 0.05227439105510712
Epoch 14 / 200 | iteration 130 / 171 | Total Loss: 3.6770365238189697 | KNN Loss: 3.6193835735321045 | CLS Loss: 0.05765296518802643
Epoch 14 / 200 | iteration 140 / 171 | Total Loss: 3.7052364349365234 | KNN Loss: 3.636918306350708 | CLS Loss: 0.06831815838813782
Epoch 14 / 200 | iteration 150 / 171 | Total Loss: 3.679098129272461 | KNN Loss: 3.6211349964141846 | CLS Loss: 0.057963162660598755
Epoch 14 / 200 | iteration 160 / 171 | Total Loss: 3.650550603866577 | KNN Loss: 3.6043601036071777 | CLS Loss: 0.04619060084223747
Epoch 14 / 200 | iteration 170 / 171 | Total Loss: 3.611435890197754 | KNN Loss: 3.586193084716797 | CLS Loss: 0.025242773815989494
Epoch: 014, Loss: 3.6855, Train: 0.9826, Valid: 0.9795, Best: 0.9797
Epoch 15 / 200 | iteration 0 / 171 | Total Loss: 3.6574466228485107 | KNN Loss: 3.5909981727600098 | CLS Loss: 0.06644848734140396
Epoch 1

...

Epoch 186 / 200 | iteration 10 / 171 | Total Loss: 3.5789926052093506 | KNN Loss: 3.5747244358062744 | CLS Loss: 0.004268075339496136
Epoch 186 / 200 | iteration 20 / 171 | Total Loss: 3.6802499294281006 | KNN Loss: 3.6704108715057373 | CLS Loss: 0.009839098900556564
Epoch 186 / 200 | iteration 30 / 171 | Total Loss: 3.640944242477417 | KNN Loss: 3.633958339691162 | CLS Loss: 0.006985971704125404
Epoch 186 / 200 | iteration 40 / 171 | Total Loss: 3.641639471054077 | KNN Loss: 3.6195478439331055 | CLS Loss: 0.02209164947271347
Epoch 186 / 200 | iteration 50 / 171 | Total Loss: 3.5816233158111572 | KNN Loss: 3.578193426132202 | CLS Loss: 0.0034298275131732225
Epoch 186 / 200 | iteration 60 / 171 | Total Loss: 3.5820138454437256 | KNN Loss: 3.576108932495117 | CLS Loss: 0.005904989317059517
Epoch 186 / 200 | iteration 70 / 171 | Total Loss: 3.6609060764312744 | KNN Loss: 3.656869888305664 | CLS Loss: 0.004036185797303915
Epoch 186 / 200 | iteration 80 / 171 | Total Loss: 3.561441898345947

Epoch 189 / 200 | iteration 70 / 171 | Total Loss: 3.600261926651001 | KNN Loss: 3.5846450328826904 | CLS Loss: 0.015616881661117077
Epoch 189 / 200 | iteration 80 / 171 | Total Loss: 3.6257970333099365 | KNN Loss: 3.6098709106445312 | CLS Loss: 0.015926232561469078
Epoch 189 / 200 | iteration 90 / 171 | Total Loss: 3.6277945041656494 | KNN Loss: 3.61649489402771 | CLS Loss: 0.01129960548132658
Epoch 189 / 200 | iteration 100 / 171 | Total Loss: 3.6359384059906006 | KNN Loss: 3.6051104068756104 | CLS Loss: 0.03082795813679695
Epoch 189 / 200 | iteration 110 / 171 | Total Loss: 3.627716302871704 | KNN Loss: 3.6101317405700684 | CLS Loss: 0.017584552988409996
Epoch 189 / 200 | iteration 120 / 171 | Total Loss: 3.651902914047241 | KNN Loss: 3.629162549972534 | CLS Loss: 0.022740429267287254
Epoch 189 / 200 | iteration 130 / 171 | Total Loss: 3.6467716693878174 | KNN Loss: 3.6378331184387207 | CLS Loss: 0.008938651531934738
Epoch 189 / 200 | iteration 140 / 171 | Total Loss: 3.625596046447

Epoch 192 / 200 | iteration 130 / 171 | Total Loss: 3.605968475341797 | KNN Loss: 3.589134454727173 | CLS Loss: 0.01683402806520462
Epoch 192 / 200 | iteration 140 / 171 | Total Loss: 3.5807862281799316 | KNN Loss: 3.576380729675293 | CLS Loss: 0.0044055818580091
Epoch 192 / 200 | iteration 150 / 171 | Total Loss: 3.5892229080200195 | KNN Loss: 3.5634138584136963 | CLS Loss: 0.02580908313393593
Epoch 192 / 200 | iteration 160 / 171 | Total Loss: 3.624157667160034 | KNN Loss: 3.6008410453796387 | CLS Loss: 0.023316603153944016
Epoch 192 / 200 | iteration 170 / 171 | Total Loss: 3.6173601150512695 | KNN Loss: 3.596792697906494 | CLS Loss: 0.020567480474710464
Epoch: 192, Loss: 3.6088, Train: 0.9963, Valid: 0.9871, Best: 0.9885
Epoch 193 / 200 | iteration 0 / 171 | Total Loss: 3.617873191833496 | KNN Loss: 3.600017547607422 | CLS Loss: 0.017855700105428696
Epoch 193 / 200 | iteration 10 / 171 | Total Loss: 3.620344877243042 | KNN Loss: 3.585192918777466 | CLS Loss: 0.03515198081731796
Epo

Epoch 196 / 200 | iteration 10 / 171 | Total Loss: 3.56596040725708 | KNN Loss: 3.5565896034240723 | CLS Loss: 0.009370706044137478
Epoch 196 / 200 | iteration 20 / 171 | Total Loss: 3.569096565246582 | KNN Loss: 3.5610709190368652 | CLS Loss: 0.00802560430020094
Epoch 196 / 200 | iteration 30 / 171 | Total Loss: 3.7674403190612793 | KNN Loss: 3.7329766750335693 | CLS Loss: 0.034463535994291306
Epoch 196 / 200 | iteration 40 / 171 | Total Loss: 3.6068832874298096 | KNN Loss: 3.5955653190612793 | CLS Loss: 0.011317950673401356
Epoch 196 / 200 | iteration 50 / 171 | Total Loss: 3.605165481567383 | KNN Loss: 3.593226432800293 | CLS Loss: 0.011938974261283875
Epoch 196 / 200 | iteration 60 / 171 | Total Loss: 3.58068585395813 | KNN Loss: 3.57743501663208 | CLS Loss: 0.003250734182074666
Epoch 196 / 200 | iteration 70 / 171 | Total Loss: 3.6278390884399414 | KNN Loss: 3.586243152618408 | CLS Loss: 0.041595980525016785
Epoch 196 / 200 | iteration 80 / 171 | Total Loss: 3.6637308597564697 | K

Epoch 199 / 200 | iteration 80 / 171 | Total Loss: 3.5880959033966064 | KNN Loss: 3.5725131034851074 | CLS Loss: 0.015582882799208164
Epoch 199 / 200 | iteration 90 / 171 | Total Loss: 3.620037078857422 | KNN Loss: 3.5903987884521484 | CLS Loss: 0.029638340696692467
Epoch 199 / 200 | iteration 100 / 171 | Total Loss: 3.633448362350464 | KNN Loss: 3.6003639698028564 | CLS Loss: 0.03308449313044548
Epoch 199 / 200 | iteration 110 / 171 | Total Loss: 3.605926513671875 | KNN Loss: 3.601919174194336 | CLS Loss: 0.004007227253168821
Epoch 199 / 200 | iteration 120 / 171 | Total Loss: 3.648345708847046 | KNN Loss: 3.6371028423309326 | CLS Loss: 0.011242974549531937
Epoch 199 / 200 | iteration 130 / 171 | Total Loss: 3.5967555046081543 | KNN Loss: 3.5879805088043213 | CLS Loss: 0.00877506285905838
Epoch 199 / 200 | iteration 140 / 171 | Total Loss: 3.622724771499634 | KNN Loss: 3.608492374420166 | CLS Loss: 0.014232449233531952
Epoch 199 / 200 | iteration 150 / 171 | Total Loss: 3.611444473266

In [8]:
test(model, test_data_iter, device)

tensor(0.9865)

In [9]:
plt.figure()
plt.plot(losses, label='train loss')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
plt.figure()
plt.plot(train_accs, label='train accuracy')
plt.plot(val_accs, label='validation accuracy')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
test_samples = torch.tensor([])
projections = torch.tensor([])
labels = torch.tensor([])

with torch.no_grad():
    for x, y in tqdm(test_data_iter):
        test_samples = torch.cat([test_samples, x])
        labels = torch.cat([labels, y])
        x = x.to(device)
        _, interm = model(x, True)
        projections = torch.cat([projections, interm.detach().cpu().flatten(1)])

In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
res = []
for i in np.arange(0.5, 4, 0.1):
    clusters = DBSCAN(eps=i, min_samples=10).fit_predict(projections)
    print(f"Number of inliers: {sum(clusters != -1) / len(clusters)}")
    res.append(sum(clusters != -1) / len(clusters))

Number of inliers: 0.618564706957197
Number of inliers: 0.6916084235530583
Number of inliers: 0.7413548947055868
Number of inliers: 0.7792243387693573
Number of inliers: 0.8090996299849254
Number of inliers: 0.8321227901877484
Number of inliers: 0.8487506281120095
Number of inliers: 0.865743913023617
Number of inliers: 0.881138367365584
Number of inliers: 0.8950710337581654
Number of inliers: 0.9076332739481979
Number of inliers: 0.9195103010369559
Number of inliers: 0.9304737106573477
Number of inliers: 0.939792608834681
Number of inliers: 0.9477867616828833
Number of inliers: 0.9554154675437394
Number of inliers: 0.9634096203919419
Number of inliers: 0.9691197295692293
Number of inliers: 0.9738705404047325
Number of inliers: 0.9789411173541638
Number of inliers: 0.9821387784934448
Number of inliers: 0.9841944177972682
Number of inliers: 0.9866155040884381
Number of inliers: 0.9886711433922617
Number of inliers: 0.990407016582157
Number of inliers: 0.9923712941391439
Number of inliers

In [26]:
plt.figure()
plt.plot(np.arange(0.5, 4, 0.1), res)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
clusters = DBSCAN(eps=1.5, min_samples=25).fit_predict(projections)

In [37]:
print(f"Number of inliers: {sum(clusters != -1) / len(clusters)}")

Number of inliers: 0.8923758622264858


In [38]:
perplexity = 100
p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [39]:
tree_dataset = list(zip(test_samples.flatten(1)[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [40]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [41]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [46]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 400
log_interval = 10
use_cuda = device != 'cpu'

In [47]:
tree = SDT(input_dim=test_samples.shape[2], output_dim=len(set(clusters)) - 1, depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [48]:
losses = []
accs = []
sparsity = []

In [ ]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=1)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
layer 5: 0.0
layer 6: 0.0
layer 7: 0.0
layer 8: 0.0
Epoch: 00 | Batch: 000 / 039 | Total loss: 2.504 | Reg loss: 0.011 | Tree loss: 2.504 | Accuracy: 0.027344 | 7.191 sec/iter
Epoch: 00 | Batch: 010 / 039 | Total loss: 2.358 | Reg loss: 0.012 | Tree loss: 2.358 | Accuracy: 0.179688 | 6.568 sec/iter
Epoch: 00 | Batch: 020 / 039 | Total loss: 2.228 | Reg loss: 0.014 | Tree loss: 2.228 | Accuracy: 0.298828 | 6.546 sec/iter
Epoch: 00 | Batch: 030 / 039 | Total loss: 2.087 | Reg loss: 0.018 | Tree loss: 2.087 | Accuracy: 0.404297 | 6.471 sec/iter
Average sparseness: 0.9946808510638298
layer 0: 0.9946808510638298
layer 1: 0.9946808510638298
layer 2: 0.9946808510638298
layer 3: 0.9946808510638298
layer 4: 0.9946808510638298
layer 5: 0.9946808510638298
layer 6: 0.9946808510638298
layer 7: 0.9946808510638298
layer 8: 0.9946808510638298
Epoch: 01 | Batch: 000 / 039 | Total loss: 2.493 | Reg loss: 0.004 | Tre

Epoch: 10 | Batch: 020 / 039 | Total loss: 1.993 | Reg loss: 0.014 | Tree loss: 1.993 | Accuracy: 0.398438 | 6.256 sec/iter
Epoch: 10 | Batch: 030 / 039 | Total loss: 1.799 | Reg loss: 0.017 | Tree loss: 1.799 | Accuracy: 0.400391 | 6.258 sec/iter
Average sparseness: 0.9946808510638298
layer 0: 0.9946808510638298
layer 1: 0.9946808510638298
layer 2: 0.9946808510638298
layer 3: 0.9946808510638298
layer 4: 0.9946808510638298
layer 5: 0.9946808510638298
layer 6: 0.9946808510638298
layer 7: 0.9946808510638298
layer 8: 0.9946808510638298
Epoch: 11 | Batch: 000 / 039 | Total loss: 2.474 | Reg loss: 0.010 | Tree loss: 2.474 | Accuracy: 0.146484 | 6.245 sec/iter
Epoch: 11 | Batch: 010 / 039 | Total loss: 2.243 | Reg loss: 0.012 | Tree loss: 2.243 | Accuracy: 0.199219 | 6.242 sec/iter
Epoch: 11 | Batch: 020 / 039 | Total loss: 1.965 | Reg loss: 0.015 | Tree loss: 1.965 | Accuracy: 0.402344 | 6.248 sec/iter
Epoch: 11 | Batch: 030 / 039 | Total loss: 1.733 | Reg loss: 0.017 | Tree loss: 1.733 | A

Epoch: 21 | Batch: 000 / 039 | Total loss: 2.450 | Reg loss: 0.013 | Tree loss: 2.450 | Accuracy: 0.294922 | 6.206 sec/iter
Epoch: 21 | Batch: 010 / 039 | Total loss: 2.071 | Reg loss: 0.015 | Tree loss: 2.071 | Accuracy: 0.357422 | 6.21 sec/iter
Epoch: 21 | Batch: 020 / 039 | Total loss: 1.765 | Reg loss: 0.017 | Tree loss: 1.765 | Accuracy: 0.476562 | 6.208 sec/iter
Epoch: 21 | Batch: 030 / 039 | Total loss: 1.646 | Reg loss: 0.019 | Tree loss: 1.646 | Accuracy: 0.458984 | 6.207 sec/iter
Average sparseness: 0.9946808510638298
layer 0: 0.9946808510638298
layer 1: 0.9946808510638298
layer 2: 0.9946808510638298
layer 3: 0.9946808510638298
layer 4: 0.9946808510638298
layer 5: 0.9946808510638298
layer 6: 0.9946808510638298
layer 7: 0.9946808510638298
layer 8: 0.9946808510638298
Epoch: 22 | Batch: 000 / 039 | Total loss: 2.451 | Reg loss: 0.014 | Tree loss: 2.451 | Accuracy: 0.250000 | 6.199 sec/iter
Epoch: 22 | Batch: 010 / 039 | Total loss: 2.093 | Reg loss: 0.015 | Tree loss: 2.093 | Ac

Epoch: 31 | Batch: 030 / 039 | Total loss: 1.556 | Reg loss: 0.021 | Tree loss: 1.556 | Accuracy: 0.435547 | 6.129 sec/iter
Average sparseness: 0.9946808510638298
layer 0: 0.9946808510638298
layer 1: 0.9946808510638298
layer 2: 0.9946808510638298
layer 3: 0.9946808510638298
layer 4: 0.9946808510638298
layer 5: 0.9946808510638298
layer 6: 0.9946808510638298
layer 7: 0.9946808510638298
layer 8: 0.9946808510638298
Epoch: 32 | Batch: 000 / 039 | Total loss: 2.427 | Reg loss: 0.016 | Tree loss: 2.427 | Accuracy: 0.312500 | 6.123 sec/iter
Epoch: 32 | Batch: 010 / 039 | Total loss: 2.059 | Reg loss: 0.017 | Tree loss: 2.059 | Accuracy: 0.433594 | 6.12 sec/iter
Epoch: 32 | Batch: 020 / 039 | Total loss: 1.698 | Reg loss: 0.019 | Tree loss: 1.698 | Accuracy: 0.474609 | 6.118 sec/iter
Epoch: 32 | Batch: 030 / 039 | Total loss: 1.488 | Reg loss: 0.021 | Tree loss: 1.488 | Accuracy: 0.496094 | 6.115 sec/iter
Average sparseness: 0.9946808510638298
layer 0: 0.9946808510638298
layer 1: 0.994680851063

Epoch: 42 | Batch: 000 / 039 | Total loss: 2.421 | Reg loss: 0.018 | Tree loss: 2.421 | Accuracy: 0.300781 | 6.048 sec/iter
Epoch: 42 | Batch: 010 / 039 | Total loss: 1.988 | Reg loss: 0.019 | Tree loss: 1.988 | Accuracy: 0.484375 | 6.05 sec/iter
Epoch: 42 | Batch: 020 / 039 | Total loss: 1.661 | Reg loss: 0.021 | Tree loss: 1.661 | Accuracy: 0.468750 | 6.051 sec/iter
Epoch: 42 | Batch: 030 / 039 | Total loss: 1.453 | Reg loss: 0.023 | Tree loss: 1.453 | Accuracy: 0.501953 | 6.052 sec/iter
Average sparseness: 0.9946808510638298
layer 0: 0.9946808510638298
layer 1: 0.9946808510638298
layer 2: 0.9946808510638298
layer 3: 0.9946808510638298
layer 4: 0.9946808510638298
layer 5: 0.9946808510638298
layer 6: 0.9946808510638298
layer 7: 0.9946808510638298
layer 8: 0.9946808510638298
Epoch: 43 | Batch: 000 / 039 | Total loss: 2.424 | Reg loss: 0.019 | Tree loss: 2.424 | Accuracy: 0.257812 | 6.05 sec/iter
Epoch: 43 | Batch: 010 / 039 | Total loss: 2.026 | Reg loss: 0.019 | Tree loss: 2.026 | Acc

Epoch: 52 | Batch: 030 / 039 | Total loss: 1.470 | Reg loss: 0.024 | Tree loss: 1.470 | Accuracy: 0.539062 | 6.045 sec/iter
Average sparseness: 0.9946808510638298
layer 0: 0.9946808510638298
layer 1: 0.9946808510638298
layer 2: 0.9946808510638298
layer 3: 0.9946808510638298
layer 4: 0.9946808510638298
layer 5: 0.9946808510638298
layer 6: 0.9946808510638298
layer 7: 0.9946808510638298
layer 8: 0.9946808510638298
Epoch: 53 | Batch: 000 / 039 | Total loss: 2.399 | Reg loss: 0.020 | Tree loss: 2.399 | Accuracy: 0.281250 | 6.048 sec/iter
Epoch: 53 | Batch: 010 / 039 | Total loss: 1.923 | Reg loss: 0.021 | Tree loss: 1.923 | Accuracy: 0.466797 | 6.049 sec/iter
Epoch: 53 | Batch: 020 / 039 | Total loss: 1.624 | Reg loss: 0.022 | Tree loss: 1.624 | Accuracy: 0.494141 | 6.052 sec/iter
Epoch: 53 | Batch: 030 / 039 | Total loss: 1.413 | Reg loss: 0.024 | Tree loss: 1.413 | Accuracy: 0.533203 | 6.053 sec/iter
Average sparseness: 0.9946808510638298
layer 0: 0.9946808510638298
layer 1: 0.99468085106

Epoch: 63 | Batch: 000 / 039 | Total loss: 2.388 | Reg loss: 0.021 | Tree loss: 2.388 | Accuracy: 0.273438 | 6.07 sec/iter
Epoch: 63 | Batch: 010 / 039 | Total loss: 1.956 | Reg loss: 0.021 | Tree loss: 1.956 | Accuracy: 0.464844 | 6.072 sec/iter
Epoch: 63 | Batch: 020 / 039 | Total loss: 1.588 | Reg loss: 0.023 | Tree loss: 1.588 | Accuracy: 0.458984 | 6.075 sec/iter
Epoch: 63 | Batch: 030 / 039 | Total loss: 1.414 | Reg loss: 0.024 | Tree loss: 1.414 | Accuracy: 0.531250 | 6.075 sec/iter
Average sparseness: 0.9946808510638298
layer 0: 0.9946808510638298
layer 1: 0.9946808510638298
layer 2: 0.9946808510638298
layer 3: 0.9946808510638298
layer 4: 0.9946808510638298
layer 5: 0.9946808510638298
layer 6: 0.9946808510638298
layer 7: 0.9946808510638298
layer 8: 0.9946808510638298
Epoch: 64 | Batch: 000 / 039 | Total loss: 2.402 | Reg loss: 0.021 | Tree loss: 2.402 | Accuracy: 0.238281 | 6.073 sec/iter
Epoch: 64 | Batch: 010 / 039 | Total loss: 1.958 | Reg loss: 0.022 | Tree loss: 1.958 | Ac

In [ ]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

In [ ]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
# plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

# Tree Visualization

In [ ]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

# Extract Rules

# Accumulate samples in the leaves

In [ ]:
print(f"Number of patterns: {len(root.get_leaves())}")

In [ ]:
method = 'greedy'

In [ ]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

# Tighten boundaries

In [ ]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")